In [1]:
# onnx전환을 합시다


In [2]:
# 필요 module 임포트

import torch
import torch.nn as nn
import torch.onnx

import onnx
import onnxruntime
import numpy as np

In [3]:
# 신경망 커스텀 class 선언

# 함수들 만들기
class Encoder(nn.Module) :
    def __init__(self, embedding_vector) :
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_vector, freeze = True, padding_idx = 0) # 라벨값을 벡터로 바꾸는 Embedding 함수
        self.rnn = nn.LSTM(embedding_vector.shape[1], embedding_vector.shape[1], batch_first = True, bidirectional = True) #임베딩 열 크기를 받아서 임베딩 열 크기를 출력
    def forward(self, x) :
        x = self.embedding(x)
        y, hc = self.rnn(x)
        return y, hc

class Decoder(nn.Module) :
    def __init__(self, embedding_vector) :
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_vector, freeze = True, padding_idx = 0)
        self.rnn = nn.LSTM(embedding_vector.shape[1] * 3, embedding_vector.shape[1], batch_first = True, bidirectional = True)
        self.f = nn.Linear(embedding_vector.shape[1] * 4, embedding_vector.shape[0]) #RNN을 계산하고 난 뒤 벡터값을 다시 라벨값으로 바꾸는 용도, 단어 출력 용도
        self.encoder_h_context = None #encoder_h 가공값 저장할 변수
    def forward(self, encoder_output, encoder_hc, t = None) : #문장 여러개 처리하기 위해서 encoder_output.shape가 필요함
        #decoder 학습 방법 2가지 (greedy, teaching_force)
        #encoder_h 가공
        encoder_h_forward = encoder_hc[0][0:1,:,:]
        encoder_h_backward = encoder_hc[0][1:2,:,:] #encoder의 h값 반으로 쪼개기
        self.encoder_h_context = torch.concat([encoder_h_forward, encoder_h_backward], dim = -1).transpose(0,1) #peeky algorithm, 적용하기 위한 변수
        
        batch_size = encoder_output.shape[0] #문장 갯수 정하기 = encoder에서 처리한 문장 갯수
        decoder_input = torch.zeros(batch_size, 1).type(torch.long).to(encoder_output.device) #처음 넣는 단어 (0번째 단어인 padding)
        decoder_hc = encoder_hc
        decoder_output_list = [] #단어들 모아서 문장으로 만들어 출력

        for i in range(4) : #t의 문자갯수만큼 반복
            decoder_output, decoder_hc = self.forward_cal(decoder_input, decoder_hc)
            decoder_output_list.append(decoder_output)

            if t is None : #greedy (t가 없음)
                decoder_input = decoder_output.argmax(dim = -1).detach()
            else : #teaching-force (t가 있음)
                decoder_input = t[:, i:i+1]

        decoder_output_list = torch.cat(decoder_output_list, dim = 1) #for 문 종료 후 list로 모아놓은 tensor들을 tensor로 합치는 작업
        return decoder_output_list, decoder_hc, None
        
    def forward_cal(self, x, h) : #실제 신경망 계산 함수
        x = self.embedding(x) #라벨을 벡터로
        x = torch.concat([self.encoder_h_context, x], dim = -1) #peeky algorithm 적용
        output, h = self.rnn(x, h) #처음 h값을 지정하고 싶으면 x 뒤에 h값 입력하세요
        output = torch.concat([self.encoder_h_context, output], dim = -1) #peeky algorithm 적용
        output = self.f(output) #벡터 계산값을 다시 라벨값으로 변환
        return output, h

class NN(nn.Module) :
    def __init__(self, embedding_vector) :
        super().__init__()
        self.encoder = Encoder(embedding_vector)
        self.decoder = Decoder(embedding_vector)

    def forward(self, x, t = None) :
        y, hc = self.encoder(x)
        y, _, _ = self.decoder(y, hc, t)
        return y

In [4]:
# 신경망 불러오기

torch_F = torch.load("add_ai.pt", weights_only=False).to("cpu")
torch_F.eval()
print(torch_F)

NN(
  (encoder): Encoder(
    (embedding): Embedding(12, 11, padding_idx=0)
    (rnn): LSTM(11, 11, batch_first=True, bidirectional=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(12, 11, padding_idx=0)
    (rnn): LSTM(33, 11, batch_first=True, bidirectional=True)
    (f): Linear(in_features=44, out_features=12, bias=True)
  )
)


In [5]:
# encoder를 onnx로 변환
x = torch.randint(0,12, size = (1,7)).type(torch.long)
dynamic_axes = {'input' : {0 : 'b'}, 'output' : {0 : 'b'}}

torch.onnx.export(
    torch_F, #변환할 신경망
    x, #torch 신경망에 넣을 파리미터
    "add_ai.onnx", #저장할 파일 이름
    export_params = True, 
    opset_version = 10, #onnx 버전
    do_constant_folding = True,
    input_names = ["input"],
    output_names = ["output"],
    dynamic_axes = dynamic_axes
)

C:\ProgramData\anaconda3\Lib\site-packages\torch\onnx\symbolic_opset9.py:4277: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(


In [6]:
# 잘 변환되었는지 확인
onnx_encoder = onnx.load("add_ai.onnx")
onnx.checker.check_model(onnx_encoder)

In [7]:
# torch 신경망과 비교

np_x = x.numpy().astype(np.int64)

onnx_F = onnxruntime.InferenceSession("add_ai.onnx", providers =  ["CPUExecutionProvider"])

onnx_input = {onnx_F.get_inputs()[0].name : np_x}
onnx_output = onnx_F.run(None, onnx_input)

# print("output : ", onnx_output[0])
# print("h : ", onnx_output[1])
# print("c : ", onnx_output[2])

y = torch_F(x)

np.testing.assert_allclose(y.detach().numpy(), onnx_output[0], rtol=1e-03, atol=1e-05)

print(y.detach().numpy())
print(onnx_output[0])

#오류 메세지가 안뜨면 오차범위 내에서 변환이 잘 이루어 진 것입니다

[[[-37.882786   -2.4349308  -4.6686225 -18.059711  -17.61088
    -9.550879    8.047626   23.350819   31.434675   30.760828
   -46.08489    -7.0632167]
  [-34.5723    -18.158762   -4.487035   -6.8067846 -15.768299
   -25.196405  -15.717453    3.2757173  25.46129    45.680687
   -47.908478  -15.194429 ]
  [-13.131937   23.739403   16.241644    3.7718143 -12.948129
   -28.413633  -31.463524  -21.183231   -6.0235314  10.071641
    24.47707   -19.353819 ]
  [ 31.420956    9.98657     5.930812   -3.5085182 -19.001299
   -33.553288  -29.617643  -11.062666   12.912622   36.129547
    10.258417  -15.391964 ]]]
[[[-37.882786   -2.4349403  -4.6686273 -18.059715  -17.610886
    -9.550886    8.047623   23.350822   31.43469    30.760847
   -46.084896   -7.063217 ]
  [-34.57231   -18.158766   -4.4870296  -6.806773  -15.768294
   -25.1964    -15.717451    3.2757168  25.461287   45.68068
   -47.9085    -15.19443  ]
  [-13.131943   23.739407   16.241642    3.7718096 -12.948138
   -28.41364   -31.463526 